In [39]:
import imageio as iio
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D as ax3d
import normalizaImg as normaliza
import numpy as np
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA # noqa, disable flycheck warning 
import cv2


In [40]:

# Leo las imagenes de entrenamiento
orig_img = iio.imread('imgs/imagenOrigen.png')
mark_img  = iio.imread('imgs/imagenMarcada.png')




# Ahora hago lo mismo con los datos normalizados segun la 'normalizacion rgb'
# Básicamente normalizo(rgb)'/1(r+g+b)
# Divido el valor RGB de cada pixel por la suma de los tres valores
img_norm = np.rollaxis((np.rollaxis(orig_img, 2)+0.1)/(np.sum(orig_img, 2)+0.1), 0, 3)

data_red = img_norm[np.where(np.all(np.equal(mark_img, (255, 0, 0)), 2))]
data_green = img_norm[np.where(np.all(np.equal(mark_img, (0, 255, 0)), 2))]
data_blue = img_norm[np.where(np.all(np.equal(mark_img, (0, 0, 255)), 2))]


data = np.concatenate([data_red, data_green, data_blue])

target = np.concatenate([np.zeros(len(data_red[:]), dtype=int),
                             np.ones(len(data_green[:]), dtype=int),
                             np.full(len(data_blue[:]), 2, dtype=int)])

clf = NearestCentroid()
clf.fit(data, target)
#####################################################################################

#Iniciamos captura del Video A(primera escena)
capture = cv2.VideoCapture('video2017-3.avi')
#out = cv2.VideoWriter('VideoSalida.mp4', cv2.VideoWriter_fourcc(*'XVID'), 80, (320, 240))
#Clasificamos el video

while (True):
    ret, frame = capture.read()
    if ret ==True:
        cv2.imshow('Video Original', frame)
    elif ret== False: 
        break
    
    
    shape = frame.shape  # Segm all
    
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Segm all
    
    img_norm = np.rollaxis((np.rollaxis(frame_rgb, 2)+0.0)/(np.sum(frame_rgb, 2)+0.0), 0, 3)
    
    paleta = np.array([[255, 0, 0], [0, 255, 0], [0, 0, 255]], dtype=np.uint8)

    
    reshape = img_norm.reshape(shape[0]*shape[1], 3)
    labels = clf.predict(reshape)
  
    reshape_back= labels.reshape(shape[0], shape[1])
    
    segImg = paleta[reshape_back]
    
    cv2.imshow("Video con segmentación euclidea",cv2.cvtColor(segImg,cv2.COLOR_RGB2BGR))
   # out.write(cv2.cvtColor(segImg,cv2.COLOR_RGB2BGR))
    
    if cv2.waitKey(1) & 0xFF==ord('q'):#se usará para finalizar el video cuando se desee
        break
    
        
#Cierra todo
capture.release()
#out.release()
cv2.destroyAllWindows()
